In [41]:
import os
import pandas as pd
from pathlib import Path
import numpy as np

In [3]:
proj_dir="/master/nplatt/schisto_aim_panel"
results_dir=f"{proj_dir}/results"

In [4]:
info_df=pd.read_csv(f"{proj_dir}/data/platt_et_al_2025_stable1.csv", sep=",", header=0)
info_df.head()

,Library ID,Museum Accession Number (NHM) or Donor ID,Predicted Species,Mitchondrial Haplotype,Population Assignment,NCBI SRA Accession,Project Citation,Country,Locality,Latitude,...,Original life-cycle stage collected,Life-cycle stage sequenced,Num Read Pairs (1e6),Coverage,Final SNV Dataset,Discordant COX1/ITS,% S. haematobium ancestry (q),Origin,Project (Collector),Comments
0,angola_cota_57,MCF03050E0612,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907442,Herein,Angola,Source of the Cota river,-9.26186,...,cercariae,cercariae,76.6,32.96,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
1,angola_cota_58,MCF03050E0613,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907441,Herein,Angola,Source of the Cota river,-9.26186,...,cercariae,cercariae,71.6,32.83,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
2,angola_cota_59,MCF03050E0614,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907440,Herein,Angola,Source of the Cota river,-9.26186,...,cercariae,cercariae,66.5,25.61,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
3,angola_icau_60,MCF03050E0615,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907439,Herein,Angola,Icau Wando village,-8.6451,...,miracidia,miracidia,56.7,25.73,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
4,angola_icau_61,MCF03050E0615,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907438,Herein,Angola,Icau Wando village,-8.6451,...,miracidia,miracidia,75.3,30.40,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN


# Admixture

In [127]:
Path(f"{results_dir}/admixture").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/admixture")

In [128]:
%%bash

conda run -n popgen --live-stream bash -c "
    vcftools \
        --vcf ../prep_vcf/scan_autosomes_maf05_ld.vcf \
        --remove-indv margrebowiei_zambia_ERR310940 \
        --recode \
        --recode-INFO-all \
        --stdout > admixture.vcf"


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
rep_vcf/scan_autosomes_maf05_ld.vcf
	--recode-INFO-all
	--recode
	--stdout
ei_zambia_ERR310940rebowi

ted at least 2 parts in FORMAT entry: ID=PID,Number=1,Type=String,Description="Physical phasing ID information, where each unique ID within a given sample (but not across samples) connects records within a phasing group">
r,Description="Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification">
conditional reference genotype confidence, encoded as a phred quality -10*log10 p(genotype call is wrong)">
llele count in genotypes, for each ALT allele, in the same order as listed">r,Description="A
ast 2 parts in INFO entry: ID=AC,Number=A,Type=Integer,Description="Allele count in genotypes, for each ALT allele, in the same order as listed">
F,Number=A,Type=Float,Description="Allele Frequency, for each ALT allele, in the same order as listed">
Frequency, for each ALT allele, 

In [138]:
%%bash

conda run -n plink --cwd . \
    plink \
        --vcf admixture.vcf \
        --out admixture \
        --double-id \
        --recode 12 \
        --allow-extra-chr

conda run -n popgen --cwd . bcftools query -l admixture.vcf >samples.list

PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
urcell, Christopher Chang   GNU General Public License v3
Logging to admixture.log.
t:tions in effec
  --allow-extra-chr
  --double-id
  --out admixture
  --recode 12
  --vcf admixture.vcf

MB RAM detected; reserving 515643 MB for main workspace.

--vcf: 1k variants complete.
iants complete.
--vcf: 3k variants complete.
--vcf: 4k variants complete.
e.vcf: 5k variants complet
--vcf: 6k variants complete.
--vcf: 7k variants complete.
--vcf: 8k variants complete.
variants complete.
--vcf: 10k variants complete.
--vcf: 11k variants complete.
omplete.2k variants c
--vcf: 13k variants complete.
--vcf: 14k variants complete.
--vcf: 15k variants complete.
-vcf: 16k variants complete.
--vcf: 17k variants complete.
--vcf: 18k variants complete.
variants complete.
--vcf: 20k variants complete.
--vcf: 21k variants complete.
omplete.2k variants c
--vcf: 23k variants complete.
--vcf: 24k variants complete.
--vcf: 25k vari

In [139]:
%%bash

#UNSUPERVISED 
#submit jobs to the cluster
QSUB="qsub -V -cwd -S /bin/bash -q all.q "  

for K in $(seq 2 5); do
    CMD="conda run -n popgen --live-stream admixture \
        --cv=1000 \
        -j12 \
        admixture.ped \
        $K \
        >admixutre.k$K.log"

    echo $CMD | qsub -V -cwd -S /bin/bash -q all.q  -N adm.k$K -o admixture.k$K.sge.stdout -e admixture.k$K.sge.stderr -pe smp 12
done

Your job 1443404 ("adm.k2") has been submitted
Your job 1443405 ("adm.k3") has been submitted
Your job 1443406 ("adm.k4") has been submitted
Your job 1443407 ("adm.k5") has been submitted


In [228]:
max_pcs

2

# Fst

In [17]:
Path(f"{results_dir}/fst").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/fst")

In [19]:
info_df=pd.read_csv(f"{proj_dir}/data/platt_et_al_2025_stable1.csv", sep=",", header=0)
info_df.head()

,Library ID,Museum Accession Number (NHM) or Donor ID,Predicted Species,Mitchondrial Haplotype,Population Assignment,NCBI SRA Accession,Project Citation,Country,Locality,Latitude,...,Original life-cycle stage collected,Life-cycle stage sequenced,Num Read Pairs (1e6),Coverage,Final SNV Dataset,Discordant COX1/ITS,% S. haematobium ancestry (q),Origin,Project (Collector),Comments
0,angola_cota_57,MCF03050E0612,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907442,Herein,Angola,Source of the Cota river,-9.26186,...,cercariae,cercariae,76.6,32.96,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
1,angola_cota_58,MCF03050E0613,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907441,Herein,Angola,Source of the Cota river,-9.26186,...,cercariae,cercariae,71.6,32.83,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
2,angola_cota_59,MCF03050E0614,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907440,Herein,Angola,Source of the Cota river,-9.26186,...,cercariae,cercariae,66.5,25.61,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
3,angola_icau_60,MCF03050E0615,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907439,Herein,Angola,Icau Wando village,-8.6451,...,miracidia,miracidia,56.7,25.73,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
4,angola_icau_61,MCF03050E0615,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907438,Herein,Angola,Icau Wando village,-8.6451,...,miracidia,miracidia,75.3,30.40,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN


In [20]:
info_df.loc[info_df["Predicted Species"] == 'S. haematobium', "Library ID"].to_csv("shae.list", sep=",", header=False, index=False)
info_df.loc[info_df["Predicted Species"] == 'S. bovis', "Library ID"].to_csv("sbov.list", sep=",", header=False, index=False)

In [21]:
%%bash

conda run -n popgen --live-stream \
    vcftools \
        --vcf ../prep_vcf/scan_autosomes_maf05_ld.vcf \
        --weir-fst-pop shae.list \
        --weir-fst-pop sbov.list


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
rep_vcf/scan_autosomes_maf05_ld.vcf
	--weir-fst-pop shae.list
	--weir-fst-pop sbov.list
.listep shae
	--keep sbov.list

ted at least 2 parts in FORMAT entry: ID=PID,Number=1,Type=String,Description="Physical phasing ID information, where each unique ID within a given sample (but not across samples) connects records within a phasing group">
r,Description="Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification">
conditional reference genotype confidence, encoded as a phred quality -10*log10 p(genotype call is wrong)">
llele count in genotypes, for each ALT allele, in the same order as listed">r,Description="A
ast 2 parts in INFO entry: ID=AC,Number=A,Type=Integer,Description="Allele count in genotypes, for each ALT allele, in the same order as listed">
F,Number=A,Type=Float,Description="Allele Frequency, for each ALT allele, in the same order as listed">
Frequency, for

In [22]:
fst_df=pd.read_csv("out.weir.fst", sep="\t", header=0)
fst_df

,CHROM,POS,WEIR_AND_COCKERHAM_FST
0,NC_067196.1,124178,0.026728
1,NC_067196.1,124348,0.848705
2,NC_067196.1,147062,-0.015993
3,NC_067196.1,147068,0.000732
4,NC_067196.1,147073,0.044099
...,...,...,...
381496,NC_067202.1,19477842,0.084907
381497,NC_067202.1,19478168,0.973279
381498,NC_067202.1,19478217,0.325318
381499,NC_067202.1,19478231,0.010759


In [26]:
fst_df = fst_df.loc[fst_df["WEIR_AND_COCKERHAM_FST"]>0].reset_index(drop=True)
fst_df.to_csv("fst_df.csv", sep=",", header=True, index=False)
fst_df

,CHROM,POS,WEIR_AND_COCKERHAM_FST
0,NC_067196.1,124178,0.026728
1,NC_067196.1,124348,0.848705
2,NC_067196.1,147068,0.000732
3,NC_067196.1,147073,0.044099
4,NC_067196.1,147161,0.007621
...,...,...,...
343761,NC_067202.1,19477842,0.084907
343762,NC_067202.1,19478168,0.973279
343763,NC_067202.1,19478217,0.325318
343764,NC_067202.1,19478231,0.010759


In [32]:
fst_df.loc[fst_df["WEIR_AND_COCKERHAM_FST"]>0.95]

,CHROM,POS,WEIR_AND_COCKERHAM_FST
2668,NC_067196.1,2623398,0.951922
2669,NC_067196.1,2662673,0.978441
2701,NC_067196.1,2710808,0.985495
2742,NC_067196.1,2736479,0.951694
2783,NC_067196.1,2753200,0.992392
...,...,...,...
343464,NC_067202.1,19329802,0.953022
343667,NC_067202.1,19457214,0.960472
343703,NC_067202.1,19465685,0.957880
343729,NC_067202.1,19470114,0.985899
